In [26]:
import pennylane as qml
import numpy as np
# If these have already been installed, this cell can be commented out.
#!pip install tensorflow_probability==0.14.0
#!pip install pennylane-lightning[gpu]

import tensorflow as tf
import tensorflow_probability as tfp
from scipy.optimize import minimize
from scipy.stats import pearsonr   
import tensorflow_probability as tfp
from __future__ import print_function
print(__doc__)

#!pip install -U scikit-learn scipy matplotlib
#!pip install grakel
#import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from grakel.datasets import fetch_dataset
from grakel.kernels import ShortestPath
from time import time
from functools import partial
import cirq, sympy
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


In [2]:
def reindex_nodes_and_edges(node_sub, edge_sub):
    # Create a dictionary to map old node indices to new node indices
    node_map = {old_index: new_index for new_index, old_index in enumerate(node_sub)}

    # Create new node indices
    new_node_sub = list(range(len(node_sub)))

    # Create new edge indices using the node mapping
    new_edge_sub = [[node_map[edge[0]], node_map[edge[1]]] for edge in edge_sub]

    return new_node_sub, new_edge_sub
def nodembedding(node_array, edge_array, node_features_array):
  
    V = len(node_array)
    subgraphs = {}
    for node_index in range(V):
        # create a subgraph containing the current node and its neighbors
        subgraph_nodes = [node_array[node_index]]
        subgraph_edges = [edge for edge in edge_array if node_array[node_index] in edge]
        for edge in subgraph_edges:
            neighbor_index = 1 if edge[0] == node_array[node_index] else 0
            neighbor = edge[neighbor_index]
            if neighbor not in subgraph_nodes:
                # find the index of the neighbor in node_array and add it to the subgraph_nodes list
                neighbor_index = np.where(node_array == neighbor)[0][0]
                subgraph_nodes.append(node_array[neighbor_index])
        
        # extract the features for the nodes in the subgraph
        sub_features = node_features_array[np.isin(node_array, subgraph_nodes)]
        
        subgraph_nodes=np.sort(np.array(subgraph_nodes))
        subgraph_edges=np.array(subgraph_edges)
        shift = np.unique(subgraph_edges.flatten())
        shift=shift.min()
        subgraph_nodes -=shift
        subgraph_edges -=shift
        subgraph_nodes,subgraph_edges= reindex_nodes_and_edges(subgraph_nodes,subgraph_edges)
        
        subgraph = (subgraph_nodes,subgraph_edges )

        # add the subgraph and sub-features to the dictionary
        subgraphs[node_index] = {'subgraph': subgraph, 'sub_features': sub_features}
    
    return subgraphs

def graph_to_arrays(G):
    edges = np.array(list(G[0]), dtype=int)
    nodes = np.unique(edges.flatten())
    node_labels = np.array([G[1][node] for node in nodes], dtype=int)
    edge_labels = np.array([G[2][tuple(edge)] for edge in edges], dtype=int)
    shift=nodes.min()
    nodes -= shift # Shift nodes so that they start at 0
    edges -= shift # Shift edges so that they start at 0

    return nodes, edges, node_labels, edge_labels

In [3]:
def lossmapping(params,nodes,node_feat):
    loss = 0
    V = len(nodes)
    if ((params.shape[0]) - len(node_feat[0])) ** 2 > 0:
        print("The number of parameters and the feature size do not coincide")    
    D = tfp.stats.correlation(node_feat, node_feat,sample_axis=1, event_axis=0)  # Compute the correlation using tfp.stats.correlation
        
    state_vectors = []
    dev = qml.device("default.qubit", wires=1)
    @qml.qnode(dev, interface="tf",diff_method="backprop")
    def circuitdummy(node,params):
        for i in range(params.shape[0]):
            qml.RX(params[i]*node_feat[node][i], wires=[0])
           
        return qml.state()
    
    for index in range(V):
        value = (circuitdummy(index,params))
        state_vectors.append(value)
    
    D_vec = tf.Variable(tf.zeros([V, V], dtype=tf.float64))# Define a TensorFlow variable to store the computed distance matrix
    for i in range(V):
        for j in range(0, i):
            innprod = tf.linalg.adjoint(tf.reshape(state_vectors[i], (2, 1))) @ tf.reshape(state_vectors[j], (2, 1))
            corr_val = tf.math.abs(tf.math.real(innprod))
            ij = tf.math.acos(corr_val)*2/np.pi
            updates = tf.squeeze(tf.stack([ij, ij]))
            D_vec = tf.tensor_scatter_nd_update(D_vec, [[i, j], [j, i]], updates)
    for i in range(V):
        for j in range(0, i):
            loss -= tf.math.abs(D[i][j] - D_vec[i][j])
    
    return loss


We do not need the following code for the dataset mutag because the node labels are from 1 to 17, but in general it can be used to encode node attributes
in such a way that the corresponding circuit represent them

parameters =np.array(np.random.normal(scale=0.3, size=(len(node_features[0]),)))
print(parameters)
params=tf.Variable(parameters,dtype=tf.float64)# Run the optimization
#--- Define the optimizer
print("params",params)
opt = tf.keras.optimizers.SGD(learning_rate=0.03)
cost =lambda: lossmapping(params,node_list,node_features)

steps = 200

for i in range(steps):
    #print(i)
    
    #print("PARAMS",params,type(params))
    #print(i)
    opt.minimize(cost, params)
    #print("last line",params,opt.minimize(cost, params),type(opt.minimize(cost, params)) )
    #print(opt.minimize(cost, params),type(opt.minimize(cost, params)))
    #print()
    if i%50==0:
        print("COST func",cost(),type(cost),"step",i)

        energy = lossmapping(params,node_list,node_features)
        print("Ground state energy: ", energy)
        
#--- Compute the final energy
energy = lossmapping(params,node_list,node_features)
print("Ground state energy: ", energy)
print(params)
print(node_features)

In [5]:
# Loads the MUTAG dataset
PTC_FM = fetch_dataset("PTC_FM", verbose=False)
G, y = PTC_FM.data, PTC_FM.target
#print(PTC_FM)

Automatically created module for IPython interactive environment
     -------------------------------------- 665.5/665.5 kB 8.4 MB/s eta 0:00:00


In [6]:
max_nodes=0
pos=0
for i in range(len(y)):
    if len(G[i][1].keys())>max_nodes:
        max_nodes=len(G[i][1].keys())
        pos=i
print(max_nodes)


64
[2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 3] <class 'numpy.ndarray'> 16


In [7]:
max_subgraphlength=0
for i in range(len(y)):
    graph_car=graph_to_arrays(G[i])
    node_list=graph_car[0]
    edge_list=graph_car[1]
    node_features= graph_car[2]
    subgraphs=nodembedding(node_list, edge_list, node_features)
    for index in subgraphs:
        #print(index, subgraphs[index]["subgraph"][0])
        if len(subgraphs[index]["subgraph"][0])>max_subgraphlength:
            max_subgraphlength=len(subgraphs[index]["subgraph"][0])
print(max_subgraphlength)
wirestot=max_subgraphlength

5


In [31]:
def circuitfinalansatz(param_fixed,var_par, graph_node_list, graph_edges_list, graph_node_feat):
    ####Preparation of the circuit according to the encoding of the node features
    ###Inputs: param_fixed are the initialization parameters to encode the information in the nodes
    ###""    : var_par are the variational parameters for the classification problem
    V=len(graph_node_list)
    for node in range(V):
        
        #for our dataset graph_node_feat is just a single number, in general unccoment those lines>:
        #for i in range(graph_node_feat.shape[1]):

                #qml.RX(param_fixed[i]*graph_node_feat[node][i], wires=[node])
        for i in range(1):

                qml.RX(param_fixed*graph_node_feat[node], wires=[node])       
    for qubit in range(V):
        qml.RZ(var_par[3*qubit], wires=qubit)
        qml.RY(var_par[3*qubit+1], wires=qubit)
        qml.RZ(var_par[3*qubit+2], wires=qubit)
    for edge in graph_edges_list:
        #Here we can insert the edge label encoded as a number, and also account for node features as (x_i -x_j)*edge_ij as rot angle to extend the expressivity of the model
        #For the simple case hereby considered we don't do that
        qml.IsingZZ(phi=var_par[3*V],wires=[edge[0],edge[1]])
    for qubit in range(wirestot): 
        qml.CNOT(wires=[qubit, (qubit+1)%(wirestot+1)])
    for qubit in range(V):
        qml.RZ(var_par[3*V+3*qubit], wires=qubit)
        qml.RY(var_par[3*V+3*qubit+1], wires=qubit)
        qml.RZ(var_par[3*V+3*qubit+2], wires=qubit)    
  
   


In [32]:
G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=0.95, random_state=42)

print(len(y_train))
print(len(y_test))


17
332


In [ ]:

var_param=np.array(np.random.normal(scale=0.3, size=(6*max_subgraphlength+1,)))
params= tf.Variable(var_param,dtype=tf.float64)
params_init=tf.Variable(2*np.pi/17)
def traincost(params):
    vn_entropies=list()
    loss=0
    state_vectors = []
    cost_vec=[]
    dev=qml.device('default.qubit', wires=max_subgraphlength+1)
    @qml.qnode(dev, interface="tf",diff_method="backprop")
    def finalcost(subgraphs,params_init,var_param,node_list):
        for node in range(len(node_list)):
            subgraph=subgraphs[node]["subgraph"]
            sub_node, sub_edge= subgraphs[node]["subgraph"]
            sub_feat=subgraphs[node]["sub_features"]
            #wires_array = np.arange(wires)
            #print(wires_array)
            circuitfinalansatz(params_init,params, sub_node, sub_edge, sub_feat)  
        return qml.expval(qml.PauliZ(max_subgraphlength))

    for i in range(len(y_train)):
    
        graph_car=graph_to_arrays(G_train[i])
        node_list=graph_car[0]
        edge_list=graph_car[1]
        node_features= graph_car[2]
        subgraphs=nodembedding(node_list, edge_list, node_features)
        value=0
        
        value=(finalcost(subgraphs,params_init,var_param,node_list))
        
        state_vectors.append(value)

    for i in range(len(y_train)):
        cost_vec.append(-tf.math.abs(state_vectors[i]+y_train[i]))
    loss=tf.reduce_sum(cost_vec)
    return loss
opt = tf.keras.optimizers.SGD(learning_rate=0.03)
    
cost =lambda: traincost(params)

steps = 200
for i in range(steps):
    print("entering the for loop:",i)
    
    
    opt.minimize(cost, params)
    
    print("COST func",cost(),type(cost),"step",i)

  
        
        

entering the for loop; 0
COST func tf.Tensor(-16.757490687082612, shape=(), dtype=float64) <class 'function'> step 0
entering the for loop; 1
COST func tf.Tensor(-16.799979650887593, shape=(), dtype=float64) <class 'function'> step 1
entering the for loop; 2
COST func tf.Tensor(-16.769034085042716, shape=(), dtype=float64) <class 'function'> step 2
entering the for loop; 3
COST func tf.Tensor(-17.136503211237986, shape=(), dtype=float64) <class 'function'> step 3
entering the for loop; 4
